## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


#MLFLOW_TRACKING_URI = "sqlite:///mlflow.db" #uri = uniform resource identifier
MLFLOW_TRACKING_URI = "http://127.0.0.1:5001"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...

- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions.

To instantiate it we need to pass a tracking URI and/or a registry URI

In [3]:
import mlflow
print(mlflow.__version__)


2.13.0


In [5]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI) #client is an object of MlflowClient class to interact with the MLflow server

client.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1716793593311, experiment_id='1', last_update_time=1716793593311, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716792602387, experiment_id='0', last_update_time=1716792602387, lifecycle_stage='active', name='Default', tags={}>]

In [6]:
client.create_experiment(name="my-cool-experiment") #create a new experiment with python API

'2'

Let's check the latest versions for the experiment with id `1`...



In [7]:
from mlflow.entities import ViewType

# what are best runs?

runs = client.search_runs(
    experiment_ids = '1',
    filter_string  = "metrics.rmse < 7",
    run_view_type  = ViewType.ACTIVE_ONLY, # aonly active runs (=not deleted)
    max_results    = 5,                    # only top 5 runs based on rmse
    order_by       = ["metrics.rmse ASC"]
)

In [8]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 42067ac60d364c1b8956dd1b5c7bdee3, rmse: 6.7423
run id: 4cee55485b6a45528f7a2ba8af8a1510, rmse: 6.9141
run id: 5ede220a3af143f2a65e276eb3cca893, rmse: 6.9235


In [10]:
"efb5a26fc3ce4abea225a4d52dba4492" in ['42067ac60d364c1b8956dd1b5c7bdee3', '4cee55485b6a45528f7a2ba8af8a1510', '5ede220a3af143f2a65e276eb3cca893']

False

### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

##### 1. Register a new version for the experiment nyc-taxi-regressor
##### 2. Retrieve the latests versions of the model nyc-taxi-regressor and check that a new version `4` was created.
##### 3. Transition the version `4` to "Staging" and adding annotations to it.

In [9]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

- promote model:

In [17]:
run_id    = "efb5a26fc3ce4abea225a4d52dba4492"                            # copy the run id from the MLflow UI
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="ny-taxi-regressor")

Registered model 'ny-taxi-regressor' already exists. Creating a new version of this model...
2024/05/27 08:53:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ny-taxi-regressor, version 3
Created version '3' of model 'ny-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1716800020371, current_stage='None', description='', last_updated_timestamp=1716800020371, name='ny-taxi-regressor', run_id='efb5a26fc3ce4abea225a4d52dba4492', run_link='', source='mlflow-artifacts:/1/efb5a26fc3ce4abea225a4d52dba4492/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>

In [18]:
model_name = "ny-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging
version: 3, stage: None


/tmp/ipykernel_4957/152114184.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


- Transition model from one stage to another:

In [19]:
mlflow.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1716796232995, description='Ny taxi duration time predictors.', last_updated_timestamp=1716800020371, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1716796276308, current_stage='Staging', description='', last_updated_timestamp=1716796919183, name='ny-taxi-regressor', run_id='42067ac60d364c1b8956dd1b5c7bdee3', run_link='', source='mlflow-artifacts:/1/42067ac60d364c1b8956dd1b5c7bdee3/artifacts/model', status='READY', status_message='', tags={'model': 'GradientBooster'}, user_id='', version='2'>,
  <ModelVersion: aliases=[], creation_timestamp=1716800020371, current_stage='None', description='', last_updated_timestamp=1716800020371, name='ny-taxi-regressor', run_id='efb5a26fc3ce4abea225a4d52dba4492', run_link='', source='mlflow-artifacts:/1/efb5a26fc3ce4abea225a4d52dba4492/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>], name='ny-taxi-regressor', tags={}>,
 <RegisteredModel: aliases=

In [15]:
client.get_latest_versions(name='ny-taxi-regressor')

/tmp/ipykernel_4957/613867140.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.get_latest_versions(name='nyc-taxi-regressor')


[<ModelVersion: aliases=[], creation_timestamp=1716799602789, current_stage='None', description='', last_updated_timestamp=1716799602789, name='nyc-taxi-regressor', run_id='efb5a26fc3ce4abea225a4d52dba4492', run_link='', source='mlflow-artifacts:/1/efb5a26fc3ce4abea225a4d52dba4492/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>]

In [20]:
for version in client.get_latest_versions(name='ny-taxi-regressor'):
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging
version: 3, stage: None


/tmp/ipykernel_4957/4017890931.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  for version in client.get_latest_versions(name='ny-taxi-regressor'):


In [21]:
model_version = 3
new_stage    = "Staging"

client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

# check the stage of the model version in the MLflow UI!

/tmp/ipykernel_4957/2623869972.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716800020371, current_stage='Staging', description='', last_updated_timestamp=1716800093377, name='ny-taxi-regressor', run_id='efb5a26fc3ce4abea225a4d52dba4492', run_link='', source='mlflow-artifacts:/1/efb5a26fc3ce4abea225a4d52dba4492/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>

- Add a description to the model version:

In [22]:
from datetime import datetime

date = datetime.today().date()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1716800020371, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2024-05-27', last_updated_timestamp=1716800190145, name='ny-taxi-regressor', run_id='efb5a26fc3ce4abea225a4d52dba4492', run_link='', source='mlflow-artifacts:/1/efb5a26fc3ce4abea225a4d52dba4492/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the DictVectorizer that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the DictVectorizer so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.

`Note:` the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.

In [23]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [26]:
#df = read_dataframe("data/green_tripdata_2021-03.csv")
df = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet")

In [27]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.') #download the preprocessor from the run

'/home/basti/MLOps-Zoomcamp/week2/preprocessor'

In [28]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [29]:
X_test = preprocess(df, dv)

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('int64'), dtype('<U1')) -> None

In [ ]:
target = "duration"
y_test = df[target].values

In [ ]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)


In [ ]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)


In [ ]:
# transition the model version to Production
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True # archive the existing production version because we are transitioning a new one
)